In [1]:
import psycopg2 as pg
import psycopg2.extras as pgExtras
import json
import pandas as pd

In [2]:
params = {  'user': 'cristiano', 'password': 'cristiano',
            'host':'localhost', 'port':'5432', 'database':'afterqualifying'}

def cleanTable():
    conn = None
    try:
        conn = pg.connect(**params)
        cur = conn.cursor()
        
        cur.execute('delete from PLACE_HEATMAP')
        
        conn.commit()
        cur.close()

        print('Heatmap table cleaned.')

    except(Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def updatePlaceHeatmapStations():
    conn = None
    try:
        conn = pg.connect(**params)
        cur = conn.cursor()
        
        cur.execute(''' update	PLACE_HEATMAP

                        set		GEOM = ST_LineInterpolatePoint(EDGE.GEOM, STATION.POSITIONINEDGE),
                                IDEDGE_FK = EDGE.IDEDGE,
                                POSITIONINEDGE = STATION.POSITIONINEDGE

                        from	STREETSEGMENT EDGE, STATION

                        where	EDGE.IDEDGE = STATION.IDEDGE_FK and
                                STATION.IDSTATION = PLACE_HEATMAP.IDPLACE   ''')

        conn.commit()
        cur.close()

        print('Heatmap Stations updated.')
    
    except(Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    
def updatePlaceHeatmapTrips():
    conn = None
    try:
        conn = pg.connect(**params)
        cur = conn.cursor()
        
        cur.execute(''' update	PLACE_HEATMAP

                        set		GEOM = PLACE.GEOM,
                                IDEDGE_FK = PLACE.IDEDGE_FK,
                                POSITIONINEDGE = PLACE.POSITIONINEDGE

                        from	PLACE

                        where	PLACE_HEATMAP.IDPLACE = PLACE.IDPLACE   ''')

        conn.commit()
        cur.close()
        
        print('Heatmap Trips updated.')
    
    except(Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [3]:
def reopenConn(conn, cur, params):
    conn.commit()
    cur.close()
    conn.close()
    conn = pg.connect(**params)
    cur = conn.cursor()

    return conn, cur

class Place:
    currentDescription = None

    def __init__(self, idPlace, edgeId, positionInEdge, valueHeatMap):
        self.idPlace = idPlace
        self.edgeId = edgeId
        self.positionInEdge = positionInEdge
        self.valueHeatMap = valueHeatMap

def loadStationsNumberVehiclesParkingSlots(fileName):
    places = list()

    with open(fileName) as jsonFile:
        optimalSolution = json.load(jsonFile)

        for var in optimalSolution["Vars"]:
            varVTag = var["VTag"][0]
            varVTagSplit = varVTag.split('_')

            if varVTag.startswith('station') and varVTag.endswith('start'):
                amountVehicles = int(var["X"])
                if amountVehicles > 0:
                    idStation = int(varVTagSplit[1])
                    place = Place(idStation, None, None, amountVehicles)
                    places.append(place)

    return places

def storeStations(fileName, description):
    Place.currentDescription = description
    places = loadStationsNumberVehiclesParkingSlots(fileName)
    
    conn = None
    try:
        conn = pg.connect(**params)
        cur = conn.cursor()
                
        lengthToInsert = 100000
        placesToInsert = list()
        sqlInsert = 'insert into PLACE_HEATMAP (IDPLACE, DESCRIPTION, VALUEHEATMAP) values %s'
        for place in places:
            placesToInsert.append((place.idPlace, Place.currentDescription, place.valueHeatMap))
            
            if len(placesToInsert) > lengthToInsert:
                pgExtras.execute_values(cur, sqlInsert, placesToInsert)
                placesToInsert = []
                conn, cur = reopenConn(conn, cur, params)
        
        if len(placesToInsert) > 0:
            pgExtras.execute_values(cur, sqlInsert, placesToInsert)

        conn.commit()
        cur.close()

        print(description + ' inserted into Heatmap.')

    except(Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()


In [4]:
class Trip:
    def __init__(self, idTrip, expansionFactor, placeStart, placeEnd, timestampDeparture, timestampArrival, drivingDistance, drivingDuration):
        self.idTrip = idTrip
        self.expansionFactor = expansionFactor
        self.placeStart = placeStart
        self.placeEnd = placeEnd
        self.timestampDeparture = timestampDeparture
        self.timestampArrival = timestampArrival
        self.drivingDistance = drivingDistance
        self.drivingDuration = drivingDuration

def idPlaceToInt(idPlace):
    intIdPlace = int(idPlace.replace('place_', ''))

    return intIdPlace

def loadTrips():
    conn = pg.connect(**params)

    sqlQuery = '''	select	'trip_'||TRIP.IDTRIP::text as IDTRIP,
                            TRIP.TRIPEXPANSIONFACTOR,
                            'place_'||TRIP.IDPLACEDEPARTURE::text as IDPLACEDEPARTURE,
                            'place_'||TRIP.IDPLACEDESTINATION::text as IDPLACEDESTINATION,
                            TRIP.TIMESTAMPDEPARTURE,
                            TRIP.TIMESTAMPARRIVAL,
                            TRIP.DRIVINGDISTANCE,
                            TRIP.DRIVINGDURATION
                    from	TRIP
                    where   TRIP.DRIVINGDISTANCE > 500
                    '''
    dataFrameEdges = pd.read_sql_query(sqlQuery, conn)
    conn.close()

    trips = dict()
    for row in dataFrameEdges.itertuples():
        dictRow = row._asdict()
        trip = Trip(idTrip=dictRow['idtrip'],
                    expansionFactor=dictRow['tripexpansionfactor'],
                    placeStart=idPlaceToInt(dictRow['idplacedeparture']),
                    placeEnd=idPlaceToInt(dictRow['idplacedestination']),
                    timestampDeparture=dictRow['timestampdeparture'],
                    timestampArrival=dictRow['timestamparrival'],
                    drivingDistance=dictRow['drivingdistance'],
                    drivingDuration=dictRow['drivingduration'])
        trips[trip.idTrip] = trip

    return trips

def loadTripsServedStartEnd(fileName, tripsDatabase):
    tripsStart = list()
    tripsEnd = list()

    with open(fileName) as jsonFile:
        optimalSolution = json.load(jsonFile)

        for var in optimalSolution["Vars"]:
            varVTag = var["VTag"][0]
            varVTagSplit = varVTag.split('_')

            if varVTag.startswith('trip'):
                idTrip = 'trip_' + varVTagSplit[1]
                amountServed = int(var["X"])
                if amountServed > 0:
                    trip = tripsDatabase[idTrip]
                    trip.valueHeatMap = amountServed
                    if varVTagSplit[2] == 'start':
                        tripsStart.append(trip)
                    elif varVTagSplit[2] == 'end':
                        tripsEnd.append(trip)
                    
    return tripsStart, tripsEnd

def storeTrips(fileName, description, tripsDatabase):
    Place.currentDescription = description
    tripsStart, tripsEnd = loadTripsServedStartEnd(fileName, tripsDatabase)
    
    conn = None
    try:
        conn = pg.connect(**params)
        cur = conn.cursor()
                
        lengthToInsert = 100000
        placesToInsert = list()
        sqlInsert = 'insert into PLACE_HEATMAP (IDPLACE, DESCRIPTION, VALUEHEATMAP) values %s'
        for trip in tripsStart:
            placesToInsert.append((trip.placeStart, 'Start ' + Place.currentDescription, trip.valueHeatMap))
        
        for trip in tripsEnd:
            placesToInsert.append((trip.placeEnd, 'End ' + Place.currentDescription, trip.valueHeatMap))

            if len(placesToInsert) > lengthToInsert:
                pgExtras.execute_values(cur, sqlInsert, placesToInsert)
                placesToInsert = []
                conn, cur = reopenConn(conn, cur, params)
        
        if len(placesToInsert) > 0:
            pgExtras.execute_values(cur, sqlInsert, placesToInsert)

        conn.commit()
        cur.close()

        print(description + ' inserted into Heatmap.')
    
    except(Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [5]:
cleanTable()

storeStations(  fileName='Optimal Solutions/Free Floating/3000/500/70.json',
                description='Initial Vehicles Free Floating 3000 km 500 m 0.7')

storeStations(  fileName='Optimal Solutions/Free Floating/3000/500/90.json',
                description='Initial Vehicles Free Floating 3000 km 500 m 0.9')

storeStations(  fileName='Optimal Solutions/Restricted 1 Partial Floating/3000/500/90.json',
                description='Initial Vehicles Restricted 1 Partial Floating 3000 km 500 m 0.9')

storeStations(  fileName='Optimal Solutions/Restricted 2 Partial Floating/3000/500/90.json',
                description='Initial Vehicles Restricted 2 Partial Floating 3000 km 500 m 0.9')

updatePlaceHeatmapStations()

tripsDatabase = loadTrips()
storeTrips( fileName='Optimal Solutions/Free Floating/3000/500/70.json',
            description='Free Floating 3000 km 500 m 0.7',
            tripsDatabase=tripsDatabase)

storeTrips( fileName='Optimal Solutions/Free Floating/3000/500/90.json',
            description='Free Floating 3000 km 500 m 0.9',
            tripsDatabase=tripsDatabase)

storeTrips( fileName='Optimal Solutions/Restricted 1 Partial Floating/3000/500/90.json',
            description='Restricted 1 Partial Floating 3000 km 500 m 0.9',
            tripsDatabase=tripsDatabase)

storeTrips( fileName='Optimal Solutions/Restricted 2 Partial Floating/3000/500/90.json',
            description='Restricted 2 Partial Floating 3000 km 500 m 0.9',
            tripsDatabase=tripsDatabase)

updatePlaceHeatmapTrips()